###  예스 24 크롤링

In [1]:
# webdriver-manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/hu_un00/.wdm/drivers/chromedriver/mac64/121.0.6167.85/chromedriver-mac-x64/chromedriver'

In [2]:
from selenium import webdriver
browser = webdriver.Chrome()

In [3]:
# url = 'https://google.com'
# browser.get(url)

In [4]:
 from selenium.webdriver.common.by import By

# browser.find_element(By.CLASS_NAME, 'uU7dJb').text
# <div class="uU7dJb">대한민국</div>

In [5]:
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [6]:
# 1페이지의 링크 데이터 전부 수집

### 한 개의 베스트 셀러 링크 수집
#browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지  전체의 링크 데이터
#browser.find_element(By.CLASS_NAME, 'gd_name') #element: 요소
# datas = browser.find_elements(By.CLASS_NAME, 'gd_name') #elements: 리스트

# for i in datas:
#     print(i.get_attribute('href'))




In [7]:
# 3페이지 까지의 링크 데이터 전부 수집 
#https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24
#https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=2&pageSize=24
#https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24

import time 

link_list = []
for i in range(1, 4):
    print("*"*10, f'현재 {i} 페이지 수집 중 입니다.',"*"*10)

    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 1페이지  전체의 링크 데이터
    #browser.find_element(By.CLASS_NAME, 'gd_name') #element: 요소
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name') #elements: 리스트

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
    
    time.sleep(3)

print(link_list)

********** 현재 1 페이지 수집 중 입니다. **********
********** 현재 2 페이지 수집 중 입니다. **********
********** 현재 3 페이지 수집 중 입니다. **********
['https://www.yes24.com/Product/Goods/122120495', 'https://www.yes24.com/Product/Goods/123675187', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/124702623', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/122090360', 'https://www.yes24.com/Product/Goods/123400303', 'https://www.yes24.com/Product/Goods/122944685', 'https://www.yes24.com/Product/Goods/118579613', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes24.com/Product/Goods/124110821', 'https://www.yes24.com/Product/Goods/124112599', 'https://www.yes24.com/Product/Goods/122428190', 'https://www.yes24.com/Product/Goods/118413014', 'https://www.yes24.com/Product/Goods/119458052', 'https://www.yes24.com/Product/Goods/119562091', 'https://www.yes24.com/Product/Goods/124702

In [8]:
len(link_list) # 24 * 3

72

In [9]:

# for link in link_list:
#     # 상세 페이지로 이동
#     browser.get(link)


browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best.gd_best_tp02').text.split(" ")[2][0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best.gd_best_tp02').text.split(" ")[6][0]


In [10]:
text = '베스트 국내도서 1위 | 국내도서 1위 9주'
text.split(" ")

['베스트', '국내도서', '1위', '|', '국내도서', '1위', '9주']

In [21]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 DB에 저장 (csv)
import pymysql # pip install pymysql
import time
import re
from datetime import datetime

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='yung00420',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:

    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # 2023년 12월 25일 -> 2023-12-25
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text

        match = re.search(r'(\d+)년 (\d+)월 (\d+)일',publishing)

        if match:
            year,month,day = match.groups()
            data_obj = datetime(int(year),int(month),int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"


        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text
        #rating = rating.replace(",","")
        if len(review) == 15 : # '첫번째 리뷰어가 되어주세요.' 글자수
            rating = 0 
        # if rating == '' :
        #     rating = 0
            

        review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text
        if len(review) == 15 : # '첫번째 리뷰어가 되어주세요.' 글자수
            review = 0 
        else :
            review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
            review = int(review.replace(",",""))

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",",""))
        # if sales == '':
        #     sales = browser.find_element(By.CLASS_NAME, 'nor_price').text
        #     sales = int(sales.replace(",",""))

        price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
        price = int(price.replace(",",""))

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text

        #print('full_text :' , full_text) #베스트만 들어옴
        parts = full_text.split("|")

        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part)) #숫자만 추출
            except:
                ranking = 0

            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
            except:
                ranking_weeks = 0
            
        # ranking = browser.find_element(By.CLASS_NAME, 'gd_best.gd_best_tp02').text.split(" ")[2][0]
        # ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best.gd_best_tp02').text.split(" ")[6][0]



        sql = """INSERT INTO Books(
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES(
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """

        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks ))
        conn.commit()

        time.sleep(2)


In [12]:
# 상세 페이지 이동 후 데이터 크롤링

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best.gd_best_tp02').text.split(" ")[2][0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best.gd_best_tp02').text.split(" ")[6][0]

title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks

('좋은 일이 오려고 그러나 보다 (리커버판)',
 '박여름 저',
 '히읏',
 '2023년 07월 26일',
 '9.5',
 '50',
 '165,591',
 '15,120',
 '7',
 '2')